# Getting Started with LangChain and OpenAI: A Quickstart Guide

This notebook provides a quick introduction to building applications with LangChain, focusing on its core components and integrating with OpenAI models. We'll also touch upon LangSmith for application tracing and observability, and LangServe for deploying your applications.

## What You Will Learn:

1.  **LangChain Setup**: How to configure your environment with LangChain, OpenAI, and LangSmith.
2.  **Core LangChain Components**: Understanding and using:
    * **Prompt Templates**: Structured ways to define inputs for LLMs.
    * **Models**: Interacting with Large Language Models (LLMs) like OpenAI's `gpt-4o`.
    * **Output Parsers**: Extracting structured information from LLM responses.
3.  **Building Simple Chains**: Combining components to create basic LLM applications.
4.  **LangSmith Tracing**: How to observe and debug your LangChain application's execution.
5.  **LangServe (Conceptual)**: An introduction to serving your LangChain applications.

## Key Concepts:

* **LangChain**: A framework for developing applications powered by language models. It provides tools to chain together different components, making it easier to build complex LLM workflows.
* **Prompt Templates**: Predefined recipes for generating prompts to LLMs. They allow for dynamic insertion of input variables.
* **Models**: The core LLMs (e.g., OpenAI's GPT models, Google's Gemini, Anthropic's Claude) that process text and generate responses.
* **Output Parsers**: Tools to structure the free-form text output from an LLM into a more usable format (e.g., string, JSON, specific data structures).
* **Chains (`Runnable`s)**: Sequences of components (prompts, models, parsers, etc.) linked together to perform a specific task. LangChain's "Runnable" protocol makes these components easily composable using the `|` operator.
* **LangSmith**: An observability and evaluation platform for LLM applications. It helps in debugging, testing, and monitoring your LangChain applications by providing detailed traces of each step.
* **LangServe**: A library for deploying LangChain runnables as REST APIs, making it easy to expose your LLM applications.

## Setup: Environment Variables

We will load API keys for OpenAI and LangChain (LangSmith) from a `.env` file for security and ease of management. Ensure you have your `.env` file correctly set up as described above.

#### Getting started With Langchain And Open AI

In this quickstart we'll see how to:

- Get setup with LangChain, LangSmith and LangServe
- Use the most basic and common components of LangChain: prompt templates, models, and output parsers.
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file.
# This makes environment variables defined in .env accessible in the script.
load_dotenv()

# Set OpenAI API key from environment variables.
# This is crucial for authentication with OpenAI's models.
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# --- LangSmith Tracking Setup ---
# Set LangChain API key for LangSmith.
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
# Enable LangSmith tracing. Setting this to "true" automatically sends traces to LangSmith.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# Define the project name under which traces will be organized in LangSmith.
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# Import ChatOpenAI from LangChain's OpenAI integration
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model.
# We're using "gpt-4o" which is a powerful, multimodal model.
llm = ChatOpenAI(model="gpt-4o")

# Print the LLM object to confirm its initialization.
print("--- Initialized LLM ---")
print(llm)

### Invoking the LLM Directly

The simplest way to interact with an LLM is to invoke it directly with a prompt string.

In [11]:
# Invoke the LLM with a simple question.
# The 'invoke' method sends the prompt to the LLM and returns its response.
result = llm.invoke("What is generative AI?")

# Print the entire response object received from the LLM.
# This object contains not just the content but also metadata like token usage, stop reasons, etc.
print("--- Direct LLM Invocation Result ---")
print(result)

# Accessing just the content of the response.
print("\n--- Content of the LLM Response ---")
print(result.content)

--- Direct LLM Invocation Result ---
content='Generative AI refers to a category of artificial intelligence systems designed to generate new content or data. This involves creating models that can produce text, images, music, or other media based on input data they have been trained on. Generative AI utilizes machine learning techniques, particularly deep learning, to understand patterns in the input data and create novel outputs that resemble the training data.\n\nOne of the most well-known types of generative AI models are Generative Adversarial Networks (GANs), which consist of two competing networks—a generator that creates new data instances, and a discriminator that evaluates them for authenticity. Other notable techniques include Variational Autoencoders (VAEs) and transformer models like GPT (Generative Pre-trained Transformer), which excel in generating human-like text.\n\nGenerative AI has numerous applications, including creating realistic images, generating art, composing m

### Chat Prompt Template

For more structured and robust interactions, especially with chat models, `ChatPromptTemplate` is used. It allows you to define messages with specific roles (system, user, assistant).

In [12]:
# Import ChatPromptTemplate for creating structured chat prompts
from langchain_core.prompts import ChatPromptTemplate

# Define a ChatPromptTemplate.
# It consists of a list of message tuples: (role, content).
# "{input}" is a placeholder that will be filled in when the prompt is invoked.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI Engineer. Provide me answers based on the questions"),
        ("user", "{input}")
    ]
)

# Print the prompt template object.
print("--- Chat Prompt Template ---")
print(prompt)

--- Chat Prompt Template ---
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answers based on the questions'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


### Building a Chain (Prompt | Model)

LangChain's core power lies in chaining components together. Here, we create a simple chain by piping the `prompt` to the `llm` using the `|` operator.

In [13]:
# Create a chain by piping the prompt template to the LLM.
# This means the output of the prompt (formatted messages) becomes the input to the LLM.
chain = prompt | llm

# Invoke the chain with the input for the placeholder defined in the prompt.
# The result is a standard LLM response object.
response = chain.invoke({"input": "Can you tell me about Langsmith?"})

# Print the full response object.
print("--- Chain Invocation Result (Raw Response) ---")
print(response)

# Print the type of the response, which is typically a LangChain message object.
print(f"\nType of response: {type(response)}")

--- Chain Invocation Result (Raw Response) ---
content='Langsmith is a tool associated with LangChain, providing capabilities for tracing and debugging in the context of working with language models. It allows developers to better understand and refine the interactions between components in applications that use large language models. By visualizing and analyzing how data flows through the system, Langsmith helps identify inefficiencies, optimize performance, and ensure that the application behaves as expected. Langsmith’s features make it a useful tool for developers looking to enhance the reliability and efficiency of language model applications.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 33, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

### Using an Output Parser (StrOutputParser)

LLMs return full response objects, which include content and metadata. Often, you just need the generated text content. `StrOutputParser` extracts this content as a simple string.

In [14]:
# Import StrOutputParser to extract the string content from LLM responses
from langchain_core.output_parsers import StrOutputParser

# Initialize the string output parser.
output_parser = StrOutputParser()

# Create a new chain: prompt -> llm -> output_parser.
# The output parser now processes the LLM's response to give a simple string.
chain = prompt | llm | output_parser

# Invoke the new chain with the same input.
# The 'response' variable will now directly contain the string output.
response_string = chain.invoke({"input": "Can you tell me about Langsmith?"})

# Print the response, which is now a plain string.
print("--- Chain Invocation Result (String Output) ---")
print(response_string)

# Verify the type of the response.
print(f"\nType of response: {type(response_string)}")

--- Chain Invocation Result (String Output) ---
Langsmith is a suite of developer tools created by LangChain, designed to enhance the development, testing, and deployment of applications built with large language models (LLMs). It offers features for tracing, debugging, testing, monitoring, and evaluating LLM-based applications. Langsmith helps developers understand how their language model applications work, by providing insights and detailed analytics about application behavior.

This toolset is particularly useful for businesses and developers who need to train and run their own custom models or applications using LLM services. Langsmith emphasizes ensuring these applications are reliable, performant, and capable of delivering consistent results. It also includes an evaluation framework with both quantitative and qualitative measures to check the performance of LLMs or chains, focused on producing effective and accurate outcomes.

Type of response: <class 'str'>
